In [2]:
import spacy,pickle,faiss,re
from spacy.tokens import Doc
from rapidfuzz import process, fuzz
from tqdm.auto import tqdm 
from sklearn.model_selection import train_test_split
import numpy as np
import polars as pl

columns_to_read = ["city_name", "district_name", "quarter_name", "street_name"]
column_types = {
    "city_name": pl.Categorical,
    "district_name": pl.Categorical,
    "quarter_name": pl.Categorical
}

base_df = pl.read_csv(
    "./data/base_df_filtered.csv",
    columns=columns_to_read,  # Sadece bu sütunları diskten oku (ÇOK VERİMLİ)
    schema_overrides=column_types      # Okunan sütunların tiplerini anında ayarla
)

nlp = spacy.load("./ner-model/ner-model-best", disable=["parser", "tagger", "lemmatizer"])

train_df = pl.read_csv("./data/train.csv")

/Users/kutaydogan/Documents/GitHub/AI-Supported-Address-Resolution/.pixi/envs/default/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kutaydogan/Documents/GitHub/AI-Supported-Address-Resolution/.pixi/envs/default/lib/python3.12/site-packages/spacy/util.py:922: UserWarning: [W095] Model 'tr_pipeline' (0.0.0) was trained with spaCy v3.4.4 and may not be 100% compatible with the current version (3.8.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
VECTORIZER_PATH = "vectorizer/street_vectorizer.pkl"
INDEX_PATH = "vectorizer/street_index_ivfpq.faiss"
STREET_LIST_PATH = "vectorizer/all_streets_list.pkl"

print("Kaydedilmiş arama varlıkları yükleniyor...")
try:
    with open(VECTORIZER_PATH, 'rb') as f:
        vectorizer = pickle.load(f)

    with open(STREET_LIST_PATH, 'rb') as f:
        all_streets_list = pickle.load(f)

    index = faiss.read_index(INDEX_PATH)
    print("Varlıklar başarıyla yüklendi.")
    print(f"İndeks Tipi: {type(index)}")
    print(f"İndeksteki Toplam Sokak Sayısı: {index.ntotal}")

except FileNotFoundError:
    print("HATA: Gerekli arama dosyaları bulunamadı. Lütfen önce Faz 1 kurulum script'ini çalıştırın.")
    # Script'i burada durdurmak iyi bir fikir olabilir.
    exit()


# --- Arama Performans Ayarı: nprobe ---
# Bu, FAISS'in arama yaparken kaç tane "klasöre" (centroid'e) bakacağını belirler.
# Değer ne kadar yüksekse, arama o kadar hassas ama bir o kadar yavaş olur.
# Değer ne kadar düşükse, arama o kadar hızlı ama çok nadir de olsa en iyi sonucu kaçırma riski olur.
# Genellikle nlist'in %1'i civarında bir değer iyi bir başlangıçtır.
index.nprobe = 10 
print(f"FAISS arama hassasiyeti (nprobe) {index.nprobe} olarak ayarlandı.")

Kaydedilmiş arama varlıkları yükleniyor...
Varlıklar başarıyla yüklendi.
İndeks Tipi: <class 'faiss.swigfaiss.IndexIVFPQ'>
İndeksteki Toplam Sokak Sayısı: 71809
FAISS arama hassasiyeti (nprobe) 10 olarak ayarlandı.


In [3]:
ALL_CITIES = set(base_df.get_column("city_name").drop_nulls().unique().to_list())
ALL_DISTRICTS = set(base_df.get_column("district_name").drop_nulls().unique().to_list())
ALL_QUARTERS = set(base_df.get_column("quarter_name").drop_nulls().unique().to_list())
ALL_STREETS = set(base_df.get_column("street_name").drop_nulls().unique().to_list())

In [4]:
turkish_map = str.maketrans("ğüşöçıİ", "gusocii")
def normalize(text: str) -> str:
    if not text:
        return ""
    return str(text).lower().translate(turkish_map).replace("i̇","i").strip()

key_to_column_map = {
    'il': 'city_name',
    'ilce': 'district_name',
    'mahalle': 'quarter_name',
    'sokak': 'street_name'
}


# Bu desen, bir metnin sonundaki adres anahtar kelimelerini hedefler.
# Sadece kelimeleri değil, başındaki boşluğu ve sonundaki ekleri de ('NDA, 'NE vb.) yakalar.
name_extractor_pattern = re.compile(
    r"\s+\b("  # Anahtar kelimeden önceki boşluğu yakala
    r"MAHALLESİ|MAHALLE|MAH|"
    r"CADDESİ|CADDE|CAD|CD|"
    r"SOKAĞI|SOKAK|SOK|SK|"
    r"BULVARI|BULVAR|BLV|"
    r"MEYDANI|MEYDAN|MEYD|"
    r"APARTMANI|APT|AP|"
    r"SİTESİ|SİTE|SİT"
    r")\b\.?(?:'[A-ZİÖÜÇŞĞ]+)?\s*$",  # Sonundaki olası nokta, 'NDA gibi ekler ve boşlukları yakala
    flags=re.IGNORECASE | re.UNICODE
)

def extract_name_part(text):
    """
    Bir adres bileşeninin sonundaki anahtar kelimeleri silerek
    sadece özel ismi döndürür.
    """
    if not isinstance(text, str):
        return text
        
    # Deseni metinle eşleştir ve eşleşen kısmı boşlukla değiştir.
    cleaned_text = name_extractor_pattern.sub("", text)
    
    # Sonuçta kalabilecek baş/son boşluklarını temizle.
    return cleaned_text.strip()


def ent2dict(doc: Doc) -> dict:
    """
    Converts entities from a spaCy Doc object into a
    predefined dictionary structure.
    
    Args:
        doc (spacy.tokens.Doc): The processed document object from spaCy.
        
    Returns:
        dict: A dictionary populated with the entity text.
    """
    entity_data = {
    'il': None,
    'ilce': None,
    'mahalle': None,
    'sokak': None,
    'semt': None, 
    'pk': None,  
    'diger': None
}
    other_entities = []
    for ent in doc.ents:
        label = ent.label_.lower()
        if label in entity_data:
            entity_data[label] = extract_name_part(normalize(ent.text))
        else:
            other_entities.append(f"{normalize(ent.text)} ({ent.label_})")
    if other_entities:
        entity_data['other'] = other_entities
        
    return entity_data


In [5]:
#trusted_entity_cache = {}
def find_trusted_entity_cached(query: str, choices: set, entity_type: str, trust_threshold: float = 90, scorer = fuzz.token_set_ratio) -> str | None:
    if not query:
        return None
    
    #cache_key = f"{entity_type}_{query}_{scorer.__name__}"
    # if cache_key in trusted_entity_cache:
    #     return trusted_entity_cache[cache_key]
    
    if query in choices:
        result = query
    else:
        extracted_match = process.extractOne(query, choices, scorer=scorer)
        result = None
        if extracted_match and extracted_match[1] >= trust_threshold:
            result = extracted_match[0]

    #trusted_entity_cache[cache_key] = result
    return result

# --- YENİ, EN GELİŞMİŞ ve EN KAPSAYICI Regex Deseni ---
# Artık şunları da tanır: 
# - Bölü (/) veya tire (-) işaretleri
# - Bu işaretlerin etrafındaki olası boşluklar (\s*)
NUMERIC_STREET_PATTERN = re.compile(r'^\s*\d+(?:\s*[/-]\s*\d+)?\s*\.?\s*$')
def find_best_street_match(query_street: str, k_faiss: int = 50, score_cutoff: float = 85.0, scorer = fuzz.token_set_ratio):
    """
    Bir sokak sorgusu için FAISS ile adayları bulur ve rapidfuzz ile en iyisini seçer.

    Args:
        query_street (str): Aranan sokak adı (örn: "ataürk caddesi").
        k_faiss (int): FAISS'in bulacağı aday sayısı.
        score_cutoff (float): Kabul edilebilir minimum rapidfuzz skoru.

    Returns:
        tuple: (en_iyi_eşleşme_str, skor) veya (None, 0)
    """
    if not query_street or not isinstance(query_street, str):
        return None, 0

    # --- AŞAMA 1: FİLTRELE (FAISS ile Hızlı Aday Bulma) ---
    query_street = normalize(query_street)
    query_street_name = extract_name_part(query_street)
    
     # --- GÜNCELLENMİŞ VE EN DOĞRU KONTROL ---
    # Sokağın ana kısmının sayısal formata (örn: "1778" veya "1778/7") uyup uymadığını kontrol et.
    if NUMERIC_STREET_PATTERN.match(query_street_name):
        # Eşleşme durumunda, sondaki olası noktayı temizleyerek net sonucu al.
        # Örnek: "1778/7." girdisi "1778/7" olarak döner.
        final_numeric_street = query_street_name.rstrip('.')
        return final_numeric_street.strip(), 100.0

    query_vector = vectorizer.transform([query_street]).toarray().astype(np.float32)
    
    distances, indices = index.search(query_vector, k_faiss)
    candidate_streets = [all_streets_list[i] for i in indices[0]]
    best_match = process.extractOne(
        query_street_name,
        candidate_streets,
        scorer=scorer
    )
    
    # 5. Eğer skor, belirlediğimiz eşik değerinin üzerindeyse sonucu döndür.
    if best_match and best_match[1] >= score_cutoff:
        return best_match[0], best_match[1]
    
    # Eşik değerinin altında kalırsa veya eşleşme bulunamazsa None döndür.
    return None, 0



In [ ]:
def calculate_match_scores(df: pl.DataFrame,
                                 entity_dict: dict,
                                 mapping: dict,
                                 score_cutoff = 85,
                                 return_top: int = 0,
                                 verbose: int = 0,
                                 ) -> pl.DataFrame:
    
    scorers_to_try = [
        fuzz.token_set_ratio,       
        fuzz.ratio, 
        fuzz.token_sort_ratio,     
    ]
    
    filtered_df = pl.DataFrame()
    successful_scorer = None
    
    for scorer in scorers_to_try:
        if verbose: print(f"--- Denenen Scorer: {scorer.__name__} ---")
        
        found_il = find_trusted_entity_cached(entity_dict.get("il"), ALL_CITIES, "il", scorer=scorer,trust_threshold=score_cutoff)
        found_ilce = find_trusted_entity_cached(entity_dict.get("ilce"), ALL_DISTRICTS, "ilce", scorer=scorer, trust_threshold=score_cutoff)
        found_mahalle = find_trusted_entity_cached(entity_dict.get("mahalle"), ALL_QUARTERS, "mahalle", scorer=scorer, trust_threshold=score_cutoff)
        found_sokak,sokak_score = find_best_street_match(entity_dict.get("sokak"), scorer=scorer, score_cutoff=score_cutoff+5)

        
        temp_df = df
        if found_il: temp_df = temp_df.filter(pl.col("city_name") == found_il)
        if found_ilce: temp_df = temp_df.filter(pl.col("district_name") == found_ilce)
        if found_mahalle: temp_df = temp_df.filter(pl.col("quarter_name") == found_mahalle)
        if found_sokak: temp_df = temp_df.filter(pl.col("street_name") == found_sokak)
        
        if not temp_df.is_empty():
            if verbose: print(f"Sonuç, {scorer.__name__} ile bulundu. Filtrelenmiş satır sayısı: {temp_df.height}")
            filtered_df = temp_df 
            successful_scorer = scorer 
            break 
        else:
            if verbose: print(f"{scorer.__name__} ile sonuç bulunamadı.")

    if filtered_df.is_empty():
        if verbose: print("Tüm denemelere rağmen eşleşme bulunamadı. NER sonucuyla fallback yapılıyor.")
        fallback_data = {}
        # Not: Fallback için en son denemedeki `found_...` değişkenlerini kullanıyoruz.
        found_entities = {"il": found_il, "ilce": found_ilce, "mahalle": found_mahalle, "sokak": found_sokak}
        for dict_key, df_col_name in mapping.items():
            fallback_data[df_col_name] = found_entities.get(dict_key)
            fallback_data[f'{df_col_name}_score'] = -1.0
        fallback_data['Overall_Score'] = -1.0
        return pl.DataFrame([fallback_data])
        
  
    score_expressions = []
    score_columns_for_avg = []
    for dict_key, df_col_name in mapping.items():
        text_value = entity_dict.get(dict_key)
        score_col_name = f'{df_col_name}_score'
        score_columns_for_avg.append(score_col_name)
        if text_value:
            choices = filtered_df.get_column(df_col_name).cast(pl.String).to_list()
            scores = process.cdist([text_value], choices, scorer=successful_scorer, score_cutoff=0)[0]
            score_expressions.append(pl.Series(name=score_col_name, values=scores))
        else:
            score_expressions.append(pl.lit(0).alias(score_col_name))
            
            
    final_df = filtered_df.with_columns(score_expressions)

    if score_columns_for_avg:
        final_df = final_df.with_columns(pl.mean_horizontal(score_columns_for_avg).alias("Overall_Score"))
    else:
        final_df = final_df.with_columns(pl.lit(0).alias("Overall_Score"))
        
    final_df = final_df.sort("Overall_Score", descending=True)
        
    if return_top > 0:
        return final_df.head(return_top)
    else:
        return final_df

In [7]:

from collections import Counter # Oylama için Counter kullanacağız

def calculate_match_scores_all_scorers(df: pl.DataFrame,
                                 entity_dict: dict,
                                 mapping: dict,
                                 score_cutoff = 85,
                                 return_top: int = 0,
                                 verbose: int = 0,
                                 ) -> pl.DataFrame:
    
    scorers_to_try = [
        fuzz.token_set_ratio,       
        fuzz.ratio, 
        fuzz.token_sort_ratio,     
    ]
    
    # --- AŞAMA 1: BİLGİ TOPLAMA ---
    # Her scorer'ın her varlık için ne bulduğunu bu sözlükte toplayalım.
    found_entities = {"il": [], "ilce": [], "mahalle": [], "sokak": []}

    for scorer in scorers_to_try:
        il = find_trusted_entity_cached(entity_dict.get("il"), ALL_CITIES, "il", scorer=scorer, trust_threshold=score_cutoff)
        ilce = find_trusted_entity_cached(entity_dict.get("ilce"), ALL_DISTRICTS, "ilce", scorer=scorer, trust_threshold=score_cutoff)
        mahalle = find_trusted_entity_cached(entity_dict.get("mahalle"), ALL_QUARTERS, "mahalle", scorer=scorer, trust_threshold=score_cutoff)
        sokak, _ = find_best_street_match(entity_dict.get("sokak"), scorer=scorer, score_cutoff=score_cutoff + 5)
        
        if il: found_entities["il"].append(il)
        if ilce: found_entities["ilce"].append(ilce)
        if mahalle: found_entities["mahalle"].append(mahalle)
        if sokak: found_entities["sokak"].append(sokak)
    
    # --- AŞAMA 2: KAPSAYICI FİLTRELEME (YENİ MANTIK) ---
    # Her varlık için bulunan tüm benzersiz adayları kullanarak filtreleme yap.
    filtered_df = df
    
    # entity_dict'te var olan ve en az bir scorer tarafından bulunan varlıkları filtrele
    for dict_key, df_col_name in mapping.items():
        found_list = found_entities.get(dict_key)
        
        if entity_dict.get(dict_key) and found_list:
            # Bulunan varlıkları tekilleştir. Örn: ['bagarasi', 'yenibagarasi', 'bagarasi'] -> {'bagarasi', 'yenibagarasi'}
            unique_entities = list(set(found_list))
            
            if verbose: print(f"Filtreye ekleniyor -> {df_col_name}.is_in({unique_entities})")
            
            # Polars'ın .is_in() fonksiyonu ile 'OR' mantığında filtreleme yap
            filtered_df = filtered_df.filter(pl.col(df_col_name).is_in(unique_entities))
            
    # --- AŞAMA 3: SKORLAMA VE SONUÇLANDIRMA ---
    if filtered_df.is_empty():
        if verbose: print("Kapsayıcı filtrelemeye rağmen eşleşme bulunamadı. Fallback yapılıyor.")
        
        # HATA DÜZELTMESİ: Fallback mantığını doğru şekilde implement et
        fallback_data = {}
        # En çok oy alan sonucu (veya herhangi birini) fallback için kullanabiliriz
        for dict_key, df_col_name in mapping.items():
            found_list = found_entities.get(dict_key)
            fallback_value = Counter(found_list).most_common(1)[0][0] if found_list else None
            fallback_data[df_col_name] = fallback_value
            fallback_data[f'{df_col_name}_score'] = -1.0
        fallback_data['Overall_Score'] = -1.0
        return pl.DataFrame([fallback_data])

    # Skorlama mantığı (öncekiyle aynı, en doğru hali)
    if verbose: print(f"\nFiltreleme tamamlandı. {filtered_df.height} aday üzerinden final skorları hesaplanıyor...")
    
    score_expressions = []
    score_columns_for_avg = []
    for dict_key, df_col_name in mapping.items():
        text_value = entity_dict.get(dict_key)
        score_col_name = f'{df_col_name}_score'
        score_columns_for_avg.append(score_col_name)
        if text_value and df_col_name in filtered_df.columns:
            choices = filtered_df.get_column(df_col_name).cast(pl.String).to_list()
            all_scores = [process.cdist([text_value], choices, scorer=s)[0] for s in scorers_to_try]
            best_scores = np.max(np.array(all_scores), axis=0)
            score_expressions.append(pl.Series(name=score_col_name, values=best_scores))
        else:
            score_expressions.append(pl.lit(0).alias(score_col_name))

    final_df = filtered_df.with_columns(score_expressions)

    if score_columns_for_avg:
        final_df = final_df.with_columns(pl.mean_horizontal(score_columns_for_avg).alias("Overall_Score"))
    else:
        final_df = final_df.with_columns(pl.lit(0).alias("Overall_Score"))
        
    final_df = final_df.sort("Overall_Score", descending=True)
        
    return final_df.head(return_top) if return_top > 0 else final_df

In [8]:
def calculate_match_scores_hypothesis(df: pl.DataFrame,
                                     entity_dict: dict,
                                     mapping: dict,
                                     score_cutoff = 85,
                                     return_top: int = 0,
                                     verbose: int = 0,
                                     ) -> pl.DataFrame:
    
    scorers_to_try = [
        fuzz.token_set_ratio,       
        fuzz.ratio, 
        fuzz.token_sort_ratio,     
    ]
    
    # --- AŞAMA 1: HİPOTEZLERİ OLUŞTURMA ---
    hypotheses = []
    consolidated_found_entities = {"il": [], "ilce": [], "mahalle": [], "sokak": []}
    
    if verbose: print("--- AŞAMA 1: Her scorer için hipotezler oluşturuluyor... ---")
    for scorer in scorers_to_try:
        il = find_trusted_entity_cached(entity_dict.get("il"), ALL_CITIES, "il", scorer=scorer, trust_threshold=score_cutoff)
        ilce = find_trusted_entity_cached(entity_dict.get("ilce"), ALL_DISTRICTS, "ilce", scorer=scorer, trust_threshold=score_cutoff)
        mahalle = find_trusted_entity_cached(entity_dict.get("mahalle"), ALL_QUARTERS, "mahalle", scorer=scorer, trust_threshold=score_cutoff)
        #sokak = find_trusted_entity_cached(entity_dict.get("sokak"),ALL_STREETS,"sokak",scorer=scorer, trust_threshold=score_cutoff)
        sokak, _ = find_best_street_match(entity_dict.get("sokak"), scorer=scorer, score_cutoff=score_cutoff + 5)
        
        hypothesis = {"scorer_name": scorer.__name__, "il": il, "ilce": ilce, "mahalle": mahalle, "sokak": sokak}
        
        if any(v for k, v in hypothesis.items() if k != 'scorer_name'):
            hypotheses.append(hypothesis)
            if il: consolidated_found_entities["il"].append(il)
            if ilce: consolidated_found_entities["ilce"].append(ilce)
            if mahalle: consolidated_found_entities["mahalle"].append(mahalle)
            if sokak: consolidated_found_entities["sokak"].append(sokak)
            if verbose > 1: print(f"  -> {scorer.__name__} hipotezi: { {k:v for k,v in hypothesis.items() if v} }")

    if not hypotheses:
        if verbose: print("Hiçbir scorer anlamlı bir hipotez üretemedi. Fallback yapılıyor.")
          # --- DÜZELTME 1: İLK FALLBACK MEKANİZMASI ---
        # Sütun sırasını garantilemek için önce isimleri, sonra skorları ekliyoruz.
        fallback_data = {}
        for col_name in mapping.values():
            fallback_data[col_name] = None
        for col_name in mapping.values():
            fallback_data[f"{col_name}_score"] = -1.0 # Skorları -1 olarak ayarlayalım
        
        fallback_data['Overall_Score'] = -1.0
        return pl.DataFrame([fallback_data])
            

    # --- AŞAMA 2: HER HİPOTEZİ AYRI AYRI DEĞERLENDİRME ---
    best_result_from_all_hypotheses = pl.DataFrame()
    if verbose: print("\n--- AŞAMA 2: Her hipotez ayrı ayrı test ediliyor... ---")
    for hypo in hypotheses:
        if verbose > 1: print(f"-> Test edilen hipotez ({hypo['scorer_name']}):")
        
        temp_df = df
        if hypo.get("il"): temp_df = temp_df.filter(pl.col("city_name") == hypo["il"])
        if hypo.get("ilce"): temp_df = temp_df.filter(pl.col("district_name") == hypo["ilce"])
        if hypo.get("mahalle"): temp_df = temp_df.filter(pl.col("quarter_name") == hypo["mahalle"])
        if hypo.get("sokak"): temp_df = temp_df.filter(pl.col("street_name") == hypo["sokak"])
        
        if temp_df.is_empty():
            if verbose > 1: print(f"  Bu hipotez ile aday bulunamadı.")
            continue
            
        score_expressions = []
        main_score_columns = []
        for dict_key, df_col_name in mapping.items():
            text_value = entity_dict.get(dict_key)
            score_col_name = f'{df_col_name}_score'
            main_score_columns.append(score_col_name)
            
            if text_value and df_col_name in temp_df.columns:
                choices = temp_df.get_column(df_col_name).cast(pl.String).to_list()
                all_scores = [process.cdist([text_value], choices, scorer=s)[0] for s in scorers_to_try]
                best_scores = np.max(np.array(all_scores), axis=0)
                score_expressions.append(pl.Series(name=score_col_name, values=best_scores))
            else:
                score_expressions.append(pl.lit(0).alias(score_col_name))
        
        hypothesis_result_df = temp_df.with_columns(score_expressions).with_columns(
            pl.mean_horizontal(main_score_columns).alias("Overall_Score")
        ).sort("Overall_Score", descending=True).head(1)
        
        if verbose: print(f"  -> {hypo['scorer_name']} hipotezinin en iyi sonucu: {hypothesis_result_df.get_column('Overall_Score')[0]:.2f} skor")
        
        best_result_from_all_hypotheses = pl.concat([
            best_result_from_all_hypotheses,
            hypothesis_result_df
        ])

    # --- AŞAMA 3: EN İYİ HİPOTEZİN SONUCUNU SEÇME ---
    if best_result_from_all_hypotheses.is_empty():
        if verbose: print("\nTüm hipotezler test edildi ancak veritabanında eşleşme bulunamadı. En iyi NER tahminiyle fallback yapılıyor.")
        fallback_data = {}
        # 1. Adım: Önce sadece isim sütunlarını doldur
        for dict_key, df_col_name in mapping.items():
            found_list = consolidated_found_entities.get(dict_key)
            fallback_value = Counter(found_list).most_common(1)[0][0] if found_list else None
            fallback_data[df_col_name] = fallback_value
        
        # 2. Adım: Sonra skor sütunlarını ekle
        for dict_key, df_col_name in mapping.items():
            fallback_data[f'{df_col_name}_score'] = -1.0
            
        fallback_data['Overall_Score'] = -1.0
        return pl.DataFrame([fallback_data])
        
    if verbose: print("\n--- AŞAMA 3: Tüm hipotezler arasından en iyi sonuç seçiliyor. ---")
    
    final_df = best_result_from_all_hypotheses.sort("Overall_Score", descending=True)
        
    if return_top > 0:
        return final_df.head(return_top)
    else:
        return final_df


In [48]:
query = "MUĞLA 48 REZİDANS AVCILAR SOK NO 5  DAİRE  329"
entity_dict = ent2dict(nlp(query))
calculate_match_scores_hypothesis(base_df,entity_dict,key_to_column_map,return_top=1,verbose=0)

city_name,district_name,quarter_name,street_name,city_name_score,district_name_score,quarter_name_score,street_name_score,Overall_Score
cat,cat,cat,str,f32,i32,i32,f32,f64
"""mugla""","""milas""","""haci ilyas""","""avcilar""",100.0,0,0,100.0,50.0


In [34]:
entity_dict = ent2dict(nlp(query))
calculate_match_scores(base_df,entity_dict,key_to_column_map,return_top=1,verbose=0)

city_name,district_name,quarter_name,street_name,city_name_score,district_name_score,quarter_name_score,street_name_score,Overall_Score
cat,cat,cat,str,i32,f32,f32,f32,f64
"""denizli""","""saraykoy""","""turan""","""zahire pazari""",0,100.0,100.0,100.0,75.0


In [31]:
entity_dict = ent2dict(nlp(query))
calculate_match_scores_all_scorers(base_df,entity_dict,key_to_column_map,verbose=0)

city_name,district_name,quarter_name,street_name,city_name_score,district_name_score,quarter_name_score,street_name_score,Overall_Score
cat,cat,cat,str,i32,f32,f32,f32,f64
"""denizli""","""saraykoy""","""turan""","""zahire pazari""",0,100.0,100.0,100.0,75.0


In [ ]:
import polars as pl
from tqdm.auto import tqdm
import os

test_df = pl.read_csv("../dataset/test.csv")
# --- 1. AYARLAR ---
BATCH_SIZE = 50_000
OUTPUT_DIRECTORY = "featured_test_parquet" 
os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)

results_list = []
batch_counter = 0  # <-- 3. ÖNERİ: Temiz bir sayaç başlattık

try:
    for i, row in enumerate(tqdm(test_df.iter_rows(named=True), total=test_df.height, desc="Eğitim Seti İşleniyor")):
        
        # --- Ana işlem mantığı (değişiklik yok) ---
        address_text = row['address']
        doc = nlp(address_text)
        ent_dict = ent2dict(doc)
    
        best_match_df = calculate_match_scores_hypothesis(
            df=base_df,
            entity_dict=ent_dict,
            mapping=key_to_column_map,
            return_top=1
        )
        
        result_row = row
        if not best_match_df.is_empty():
            match_dict = best_match_df.to_dicts()[0]
            result_row.update(match_dict)
        
        if 'address' in result_row and isinstance(result_row['address'], str):
            result_row['address'] = result_row['address'].replace('\n', ' ').replace('\r', ' ')
        results_list.append(result_row)
        
        # --- Batch yazma mantığı ---
        # <-- 1. ÖNERİ: Batch koşulu (i + 1) olarak güncellendi
        if (i + 1) % BATCH_SIZE == 0:
            batch_counter += 1 # <-- 3. ÖNERİ: Sayacı artır
            print(f"\nBatch {batch_counter} diske yazılıyor...")
            
            batch_df = pl.DataFrame(results_list)
            output_path = f"{OUTPUT_DIRECTORY}/batch_{batch_counter}.parquet"
            batch_df.write_parquet(output_path)
            
            results_list = [] # Listeyi sıfırla
            print(f"Batch yazıldı ve bellek temizlendi.")

except KeyboardInterrupt:
    print("\nİşlem kullanıcı tarafından durduruldu. Kalan sonuçlar kaydediliyor.")
    # pass komutu sayesinde kod finally bloğuna devam edecek

finally:
    # <-- 2. ÖNERİ: Son kaydetme işlemini finally bloğuna aldık
    if results_list:
        batch_counter += 1 # <-- 3. ÖNERİ: Son batch için de sayacı artır
        print(f"\nDöngü bitti/durduruldu. Kalan {len(results_list)} satırlık son batch ({batch_counter}) diske yazılıyor...")
        
        batch_df = pl.DataFrame(results_list)
        output_path = f"{OUTPUT_DIRECTORY}/batch_{batch_counter}.parquet"
        batch_df.write_parquet(output_path)
        
        results_list = []
        print(f"Son batch yazıldı ve bellek temizlendi.")

    print("\nTüm işlemler tamamlandı.")